In [3]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [6]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'processed'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [13]:
ct = pd.read_csv(src / 'ct-from-test-data' / 'ct.csv', index_col='date_sample')
ct_by_awh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-awh.csv')
ct_by_adh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-adh.csv')
ct_by_sex = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-sex.csv')
ct_by_ag = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-ag.csv')
addiv = pd.read_csv(ref / 'addiv.csv')[['id_addiv', 'name_addiv_2']]
pop = pd.read_csv(ref / 'pop_1.csv')[['id_addiv', 'pop']]

In [14]:
ct

no_test  no_test_pos  no_test_rollingsum7d  \
date                                                       
2021-05-27     2965.0          2.0                   NaN   
2021-05-28     3495.0          9.0                   NaN   
2021-05-29     5083.0         21.0                   NaN   
2021-05-30     6828.0          6.0                   NaN   
2021-05-31     5514.0          8.0                   NaN   
...               ...          ...                   ...   
2021-09-24  2299354.0      11014.0             8137745.0   
2021-09-25  1817203.0       7634.0             9881665.0   
2021-09-26  2114164.0       8282.0            11896256.0   
2021-09-27  3225681.0       7685.0            15037491.0   
2021-09-28   927272.0       4616.0            15367682.0   

            no_test_rollingmean7d  no_test_pos_rollingsum7d  \
date                                                          
2021-05-27                    NaN                       NaN   
2021-05-28                    NaN                       NaN   
2021-05-29                    NaN                       NaN   
2021-05-30                    NaN                       NaN   
2021-05-31                    NaN                       NaN   
...                           ...                       ...   
2021-09-24           1.162535e+06                   69697.0   
2021-09-25           1.411666e+06                   68593.0   
2021-09-26           1.699465e+06                   68319.0   
2021-09-27           2.148213e+06                   66804.0   
2021-09-28           2.195383e+06                   61478.0   

            no_test_pos_rollingmean7d  pct_test_pos_per7d   ct  \
date                                                             
2021-05-27                        NaN                 NaN  NaN   
2021-05-28                        NaN                 NaN  NaN   
2021-05-29                        NaN                 NaN  NaN   
2021-05-30                        NaN                 NaN  NaN   
2021-05-31                        NaN                 NaN  NaN   
...                               ...                 ...  ...   
2021-09-24                9956.714286            0.008565  1.0   
2021-09-25                9799.000000            0.006941  1.0   
2021-09-26                9759.857143            0.005743  1.0   
2021-09-27                9543.428571            0.004442  1.0   
2021-09-28                8782.571429            0.004000  1.0   

            situational_level  
date                           
2021-05-27                NaN  
2021-05-28                NaN  
2021-05-29                NaN  
2021-05-30                NaN  
2021-05-31                NaN  
...                       ...  
2021-09-24                2.0  
2021-09-25                2.0  
2021-09-26                2.0  
2021-09-27                2.0  
2021-09-28                2.0  

[125 rows x 9 columns]

In [5]:
tphcm = {
    'addr_dist_home':['THANH PHO HO CHI MINH'],
    'no_test_corrected': [ct[['no_test_corrected', 'no_positive']].sum()[0]],
    'no_positive': [ct[['no_test_corrected', 'no_positive']].sum()[1]]
}

df_tphcm = pd.DataFrame(tphcm)

df = (
    ct_by_adh[['date_sample', 'addr_dist_home', 'no_test_corrected', 'no_positive']]
#     .query('date_sample == "2021-08-05"')
    .groupby('addr_dist_home')[['no_test_corrected', 'no_positive']]
    .sum()
#     .to_frame(name='no_test_cumsum')
    .reset_index()
    .append(df_tphcm)
    .sort_values('no_test_corrected')
    .merge(
        addiv,
        how = 'left',
        left_on = 'addr_dist_home',
        right_on = 'name_addiv_2'
    )
    .merge(
        pop,
        how = 'left',
        left_on = 'id_addiv',
        right_on = 'id_addiv'
    )
    .drop(columns={'id_addiv', 'name_addiv_2'})
)

df['pct_positive'] = df['no_positive'] / df['no_test_corrected']

df['predicted_inc'] = df['pop'] * df['pct_positive']

df.to_csv(src / 'predicted-inc-from-test-data' / 'predicted-inc.csv', index=False)
df

addr_dist_home  no_test_corrected  no_positive      pop  \
0           HUYEN CAN GIO            24757.0       2344.0    71526   
1                 QUAN 11            28333.0       8438.0   209867   
2          QUAN PHU NHUAN            34565.0       6998.0   163961   
3                 QUAN 06            39155.0      18656.0   233561   
4                 QUAN 01            39857.0      14871.0   142625   
5            HUYEN NHA BE            40836.0      21255.0   206837   
6                 QUAN 05            42964.0      15108.0   159073   
7                 QUAN 02            44182.0       4666.0   180275   
8                 QUAN 04            48864.0      18765.0   175329   
9            QUAN TAN PHU            66146.0      22909.0   485348   
10                QUAN 03            69140.0      12220.0   190375   
11                QUAN 07            69202.0      17828.0   360155   
12       HUYEN BINH CHANH            74178.0      23819.0   705508   
13                QUAN 10            74692.0      22006.0   234819   
14                QUAN 08            76121.0      25322.0   424667   
15                QUAN 12            82023.0      25026.0   620146   
16           HUYEN CU CHI            82551.0       9865.0   462047   
17           QUAN THU DUC            89196.0      17708.0   592686   
18        QUAN BINH THANH            92654.0      25347.0   499164   
19            QUAN GO VAP            93663.0      14702.0   676899   
20                QUAN 09            94023.0       9094.0   397006   
21          QUAN TAN BINH           100422.0      24467.0   474792   
22          HUYEN HOC MON           103626.0      22627.0   542243   
23          QUAN BINH TAN           144401.0      41222.0   784173   
24  THANH PHO HO CHI MINH          2236394.0     443631.0  8993082   

    pct_positive  predicted_inc  
0       0.094680   6.772103e+03  
1       0.297815   6.250160e+04  
2       0.202459   3.319540e+04  
3       0.476465   1.112837e+05  
4       0.373109   5.321465e+04  
5       0.520497   1.076580e+05  
6       0.351643   5.593694e+04  
7       0.105609   1.903859e+04  
8       0.384025   6.733073e+04  
9       0.346340   1.680954e+05  
10      0.176743   3.364742e+04  
11      0.257623   9.278407e+04  
12      0.321106   2.265428e+05  
13      0.294623   6.918314e+04  
14      0.332655   1.412674e+05  
15      0.305110   1.892125e+05  
16      0.119502   5.521549e+04  
17      0.198529   1.176654e+05  
18      0.273566   1.365544e+05  
19      0.156967   1.062508e+05  
20      0.096721   3.839882e+04  
21      0.243642   1.156792e+05  
22      0.218353   1.184001e+05  
23      0.285469   2.238570e+05  
24      0.198369   1.783948e+06

In [6]:
# df = (
#     ct_by_adh[['addr_dist_home', 'date_sample', 'no_test', 'no_positive']]
#     .groupby(['addr_dist_home', 'date_sample']).sum()
#     .groupby(level=0).cumsum().reset_index()
#     .merge(
#         addiv,
#         how = 'left',
#         left_on = 'addr_dist_home',
#         right_on = 'name_addiv_2'
#     )
#     .merge(
#         pop,
#         how = 'left',
#         left_on = 'id_addiv',
#         right_on = 'id_addiv'
#     )
#     .drop(columns={'id_addiv', 'name_addiv_2'})
# )

# df['pct_positive'] = df['no_positive'] / df['no_test']

# df['predicted_inc'] = df['pop'] * df['pct_positive']
# # foo[foo.addr_dist_home == 'HUYEN BINH CHANH'].tail(10)
# df